In this notebook we play a bit with the function multi_get_b_best, which computes the best block spectrum $b_{best}$ for a given vector $\boldsymbol{n} \in [0,1]^d$ and integer multipartition of $d$.

In [ ]:
import numpy as np
import torch
import sys
import os

# Add the src directory to the path to import our custom module
sys.path.insert(0, os.path.join('..', 'src'))

Let us import the functions we'll use in this notebook:
- `multi_get_b_best`computes $\boldsymbol{b}_{best}$ numerically for given $\boldsymbol{n} \in [0,1^d]$ and integer partition of the integer $d$
- `b_best_ana` computes $\boldsymbol{b}_{best}$ using exact formulas for simple integer partitions that allow for analytical treatment
- `find_conjecture`takes $\boldsymbol{n}$ and a (numerical) estimate of $\boldsymbol{b}_{best}$ as input, and returns the 'optimal' \emph{set} partition $P$ of the set $\{ 0, \ldots, d-1 \}$ such that the vector obtained by averaging entries of $\boldsymbol{n}$ as indicated by that set partition is as close as possible to $\boldsymbol{b}_{best}$. (Check out the function `avg` in `utils.py` for more on this)

In [ ]:
from kaustav_conj.core import multi_get_b_best, b_best_ana, find_conjecture

We now check that the optimizer `multi_get_b_best` does its job for $d=4$. We do so by comparing what it returns with the analytical formulas given by `b_best_ana`.

In [5]:
n_vectors = [[0.9, 0.3, 0.2, 0.1], [1., 0.7, 0.2, 0.1], [1., 0.9, 0.2, 0.1] ]
int_partitions_of_4 = list_all_integer_partitions(4)
for int_partition in int_partitions_of_4:
    print(f"\n ==== Current integer partition: {int_partition} ====\n")
    for n in n_vectors:
        b_best_analytical = b_best_ana(n,int_partition)
        print(f"Analytical result: b_best = {b_best_analytical}")
        U_best, b_best_num, H_best = multi_get_b_best(n, int_partition, N_init=1, N_steps=400,learning_rate=0.01, verbosity=0)
        print(f"Numerical result: b_best = {b_best_num}")
        print(f"Norm of difference: {np.linalg.norm(b_best_analytical - b_best_num)}\n")


 ==== Current integer partition: [3, 1] ====

Analytical result: b_best = [0.5 0.3 0.2 0.5]
Numerical result: b_best = [0.50000001 0.3        0.2        0.49999999]
Norm of difference: 8.071498967696998e-09

Analytical result: b_best = [0.7  0.55 0.2  0.55]
Numerical result: b_best = [0.7        0.55000003 0.2        0.54999997]
Norm of difference: 4.440889947872472e-08

Analytical result: b_best = [0.9  0.55 0.2  0.55]
Numerical result: b_best = [0.9  0.55 0.2  0.55]
Norm of difference: 4.306265659350951e-09


 ==== Current integer partition: [2, 2] ====

Analytical result: b_best = [0.5  0.25 0.5  0.25]
Numerical result: b_best = [0.49999999 0.24999999 0.50000001 0.25000001]
Norm of difference: 1.496628501220632e-08

Analytical result: b_best = [0.55 0.45 0.55 0.45]
Numerical result: b_best = [0.55       0.44999999 0.55       0.45000001]
Norm of difference: 1.3561830829678097e-08

Analytical result: b_best = [0.55 0.55 0.55 0.55]
Numerical result: b_best = [0.55000002 0.54999998 0.5

Next, we check that `find_conjecture` finds the correct conjecture for a simple case.

In [ ]:
n = [0.9, 0.8912, 0.8113, 0.1]
int_partition = [2,1,1]
U_best, b_best_num, H_best = multi_get_b_best(n, int_partition, N_init=1, N_steps=400,learning_rate=0.01, verbosity=0)
print(b_best_num)
find_conjecture(n, b_best_num) # should return a couple (small number, [[2], [0, 1, 3]])

[0.8113     0.63040001 0.6304     0.6304    ]


(4.694119279283327e-09, [[2], [0, 1, 3]])

We now see how we can use `find_conjecture` systematically for many vectors $\boldsymbol{n}$ to formulate general conjectures, e.g., for $d=5$ and its integer partition $[3,2]$. We know analytically that the best set partition of $\{0,1,2,3,4 \}$ in this case should be $\{ \{ 0, 4 \}, \{ 1, 3 \}, \{  2 \} \}$. The implementation is basically the same as that of `get_conjecture.py`

In [16]:
d = 5
int_partition = [3,2]
N_init = 4
N_steps = 400
learning_rate = 0.01
N_n = 5
eps = 1.e-6

import numpy as np
from kaustav_conj.core import multi_get_b_best, find_conjecture
from kaustav_conj.utils import list_all_integer_partitions
from more_itertools import set_partitions
from collections import Counter

output = [] # will contain 4-tuples [n, b_best_num, best_P, rel_err]
best_P_counter = Counter()

for _ in range(N_n):
    n = np.sort(np.random.rand(d)) # random spectrum with entries in [0,1]

     # perform optimization of our cost function
    U_best, b_best_num, H_best = multi_get_b_best(n, int_partition, N_init=N_init, N_steps=N_steps,learning_rate=learning_rate, verbosity=0)

    # get best_P, the set partition P of {0,...,d} that minimizes norm(avg(P,n) - b_best_num); and get rel_err = norm(norm(avg(best_P,n)) - b_best_num)/||n||
    rel_err, best_P = find_conjecture(n, b_best_num)

    # store output. If needed, also store best unitaries and H_best values
    out = [n, b_best_num, best_P, rel_err]
    output.append(out)
    print(out) # may want to print some of the output data

    if(rel_err > eps):
        print(f"Warning: norm of diff = {rel_err} is larger than prescribed convergence threshold eps")

    # Convert the inner lists of best_P to tuples to make them hashable, then increase by 1 best_P counter
    hashable_best_P = tuple(tuple(block) for block in best_P)
    best_P_counter[hashable_best_P] += 1

print("\nThe counts of the partitions of {{0,1,2,3,4}} are:")
print(best_P_counter)

[array([0.04870483, 0.10181971, 0.77398577, 0.82959807, 0.91333428]), array([0.77398577, 0.48101953, 0.46570883, 0.48101959, 0.46570895]), [[2], [1, 3], [0, 4]], 6.177669010832647e-08]
[array([0.11939977, 0.5431929 , 0.56697467, 0.60676019, 0.6947086 ]), array([0.57497656, 0.56697467, 0.40705418, 0.57497652, 0.4070542 ]), [[2], [1, 3], [0, 4]], 2.52036926970754e-08]
[array([0.32199733, 0.59177527, 0.74881933, 0.85859739, 0.91080932]), array([0.74881933, 0.72518633, 0.61640332, 0.72518633, 0.61640333]), [[2], [1, 3], [0, 4]], 4.696111393404994e-09]
[array([0.0997801 , 0.32607368, 0.55093887, 0.59920703, 0.86923412]), array([0.55093887, 0.48450711, 0.46264036, 0.48450711, 0.46264035]), [[2], [1, 3], [0, 4]], 1.0402101284779761e-08]
[array([0.15311853, 0.24231298, 0.27031748, 0.31656943, 0.90615322]), array([0.52963586, 0.2794412 , 0.27031748, 0.52963589, 0.27944121]), [[2], [1, 3], [0, 4]], 2.2787240089734324e-08]

The counts of the partitions of {{0,1,2,3,4}} are:
Counter({((2,), (1, 3)